In [ ]:
import gdown
import os
import pandas as pd
from google.colab import drive
# part 1
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import pickle
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/train_data.csv", low_memory=False)
title_brand = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/title_brand.csv", low_memory=False)
test_df = pd.read_csv("/content/drive/MyDrive/Datasets/Amazon/test_data.csv", low_memory=False)

## keras with asin & vote & varified & reviewerID

### پیش پردازش داده ها

In [ ]:
## پیوند دیتافریم آموزشی با دیتافریم عنوان و برند
df_1 = df_train.copy()

In [ ]:
df_1.vote = pd.to_numeric(df_1.vote, errors='coerce')
df_1.vote = df_1.vote.fillna(0.0) ##

In [ ]:
## پاک‌سازی متن
def clean_text(text):
    # تبدیل به حروف کوچک
    text = text.lower()
    # حذف علائم نگارشی و کاراکترهای خاص
    text = re.sub(r'[^\w\s]', '', text)
    # حذف فاصله‌های اضافی
    text = ' '.join(text.split())
    return text

# اعمال پاک‌سازی روی ستون reviewText
df_1['reviewText'] = df_1['reviewText'].apply(clean_text)

# 3. تبدیل برچسب‌ها (۱ تا ۵ به ۰ تا ۴)
df_1['overall'] = df_1['overall'] - 1

In [ ]:
le_asin = LabelEncoder()
le_reviewerID = LabelEncoder()
df_1['asin'] = le_asin.fit_transform(df_1['asin'])
df_1['reviewerID'] = le_asin.fit_transform(df_1['reviewerID'])

In [ ]:
## تقسیم داده های آموزشی و اعتبار سنجی
train_df, val_df = train_test_split(df_1, test_size=0.3, random_state=42)

# توکن‌سازی با توکنایزر Keras
vocab_size = 10000  # تعداد کلمات در واژگان
max_length = 512  # حداکثر طول توکن‌ها
oov_token = '<OOV>'  # توکن برای کلمات ناشناخته

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_df['reviewText'])

# تبدیل متن به توکن‌ها
train_sequences = tokenizer.texts_to_sequences(train_df['reviewText'])
val_sequences = tokenizer.texts_to_sequences(val_df['reviewText'])

In [ ]:
# آماده‌سازی ویژگی‌های اضافی
extra_features = ['asin', 'reviewerID']
train_extra = train_df[extra_features].values
val_extra = val_df[extra_features].values

### Model

In [ ]:
# تعریف مدل Keras با دو ورودی (متن + ویژگی‌های اضافی)
# ورودی ویژگی‌های اضافی
vote_input = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='vote')
asin_input = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='asin')
reviewerID_input = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='reviewerID')
verified_input = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='verified')

# تعداد دسته‌های منحصربه‌فرد برای Embedding
num_asin = len(le_asin.classes_)
num_reviewer = len(le_reviewerID.classes_)

## پردازش ویژگی های اضافی غیر ترتیبی
asin_embedding = tf.keras.layers.Embedding(input_dim=num_asin, output_dim=16)(asin_input)
reviewer_embedding = tf.keras.layers.Embedding(input_dim=num_reviewer, output_dim=16)(reviewerID_input)
asin_embedding = tf.keras.ops.squeeze(asin_embedding, axis=1)
reviewer_embedding = tf.keras.ops.squeeze(reviewer_embedding, axis=1)

# ترکیب ویژگی‌های اضافی
extra_features = tf.keras.layers.Concatenate()([vote_input, asin_embedding, reviewer_embedding, verified_input])
extra_dense = tf.keras.layers.Dense(64, activation='relu')(extra_features)

# ورودی متن
text_input = tf.keras.layers.Input(shape=(max_length,), name='text_input')
embedding = tf.keras.layers.Embedding(vocab_size, 128)(text_input)
lstm = tf.keras.layers.LSTM(64)(embedding)
text_output = tf.keras.layers.Dense(64, activation='relu')(lstm)

# ترکیب تمام ورودی‌ها
combined = tf.keras.layers.Concatenate()([text_output, extra_dense])
dense = tf.keras.layers.Dense(64, activation='relu')(combined)
output = tf.keras.layers.Dense(5, activation='softmax')(dense)

In [ ]:
# تعریف مدل
model = tf.keras.Model(inputs=[text_input, vote_input, asin_input, reviewerID_input, verified_input], outputs=output)

# کامپایل مدل
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ asin (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reviewerID          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_input          │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 16)     │  1,451,440 │ asin[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 16)     │  5,138,160 │ reviewerID[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 512, 128)  │  1,280,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vote (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ squeeze (Squeeze)   │ (None, 16)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ squeeze_1 (Squeeze) │ (None, 16)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ verified            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 34)        │          0 │ vote[0][0],       │
│ (Concatenate)       │                   │            │ squeeze[0][0],    │
│                     │                   │            │ squeeze_1[0][0],  │
│                     │                   │            │ verified[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      4,160 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      2,240 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128)       │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5)         │        325 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,933,989 (30.27 MB)

 Trainable params: 7,933,989 (30.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# آموزش مدل
history = model.fit(
    [train_padded, train_extra], train_labels,
    validation_data=([val_padded, val_extra], val_labels),
    epochs=7,
    batch_size=32,
    class_weight=class_weights
)

Epoch 1/7


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['text_input', 'vote', 'asin', 'reviewerID', 'verified']
Received: inputs=('Tensor(shape=(None, 512))', {'vote': 'Tensor(shape=(None,))', 'asin': 'Tensor(shape=(None,))', 'reviewerID': 'Tensor(shape=(None,))', 'verified': 'Tensor(shape=(None,))'})
  warnings.warn(msg)


18352/18352 ━━━━━━━━━━━━━━━━━━━━ 519s 28ms/step - accuracy: 0.5574 - loss: 10.4650 - val_accuracy: 0.6488 - val_loss: 0.8804
Epoch 2/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 515s 28ms/step - accuracy: 0.6576 - loss: 6.5207 - val_accuracy: 0.6683 - val_loss: 0.8301
Epoch 3/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 560s 28ms/step - accuracy: 0.6893 - loss: 5.8256 - val_accuracy: 0.6756 - val_loss: 0.8243
Epoch 4/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 575s 29ms/step - accuracy: 0.7072 - loss: 5.4272 - val_accuracy: 0.6877 - val_loss: 0.8223
Epoch 5/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 563s 29ms/step - accuracy: 0.7236 - loss: 5.0857 - val_accuracy: 0.6857 - val_loss: 0.8050
Epoch 6/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 525s 29ms/step - accuracy: 0.7377 - loss: 4.7425 - val_accuracy: 0.6845 - val_loss: 0.8330
Epoch 7/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 527s 29ms/step - accuracy: 0.7532 - loss: 4.4178 - val_accuracy: 0.6811 - val_loss: 0.8606


## keras without extra features

### پیش پردازش داده ها

In [ ]:
## پیوند دیتافریم آموزشی با دیتافریم عنوان و برند
df_2 = df_train.copy()

In [ ]:
## پاک‌سازی متن
def clean_text(text):
    # تبدیل به حروف کوچک
    text = text.lower()
    # حذف علائم نگارشی و کاراکترهای خاص
    text = re.sub(r'[^\w\s]', '', text)
    # حذف فاصله‌های اضافی
    text = ' '.join(text.split())
    return text

# اعمال پاک‌سازی روی ستون reviewText
df_2['reviewText'] = df_2['reviewText'].apply(clean_text)

# 3. تبدیل برچسب‌ها (۱ تا ۵ به ۰ تا ۴)
df_2['overall'] = df_2['overall'] - 1

In [ ]:
## تقسیم داده های آموزشی و اعتبار سنجی
train_df, val_df = train_test_split(df_2, test_size=0.3, random_state=42)

# توکن‌سازی با توکنایزر Keras
vocab_size = 10000  # تعداد کلمات در واژگان
max_length = 512  # حداکثر طول توکن‌ها
oov_token = '<OOV>'  # توکن برای کلمات ناشناخته

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_df['reviewText'])

# تبدیل متن به توکن‌ها
train_sequences = tokenizer.texts_to_sequences(train_df['reviewText'])
val_sequences = tokenizer.texts_to_sequences(val_df['reviewText'])

### Model

In [ ]:
# تعریف مدل Keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

# کامپایل مدل
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# نمایش خلاصه مدل
model.summary()

# آموزش مدل
history = model.fit(
    train_padded, train_labels,
    validation_data=(val_padded, val_labels),
    epochs=20,
    batch_size=32,
    class_weight=class_weights
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 443s 24ms/step - accuracy: 0.5501 - loss: 10.9924 - val_accuracy: 0.5494 - val_loss: 1.5202
Epoch 2/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 429s 23ms/step - accuracy: 0.5614 - loss: 9.5851 - val_accuracy: 0.6229 - val_loss: 0.9658
Epoch 3/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 423s 22ms/step - accuracy: 0.6364 - loss: 6.9321 - val_accuracy: 0.6543 - val_loss: 0.8474
Epoch 4/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 442s 22ms/step - accuracy: 0.6669 - loss: 6.1753 - val_accuracy: 0.6729 - val_loss: 0.8404
Epoch 5/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 463s 24ms/step - accuracy: 0.6881 - loss: 5.7187 - val_accuracy: 0.6643 - val_loss: 0.8597
Epoch 6/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 433s 24ms/step - accuracy: 0.7052 - loss: 5.3514 - val_accuracy: 0.6825 - val_loss: 0.8000
Epoch 7/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 446s 24ms/step - accuracy: 0.7193 - loss: 5.0406 - val_accuracy: 0.6800 - val_loss: 0.8328
Epoch 8/20
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 423s 23ms/

## Keras with asin & summary

### پیش پردازش داده ها

In [ ]:
## پیوند دیتافریم آموزشی با دیتافریم عنوان و برند
df_3 = df_train.copy()

In [ ]:
## پاک‌سازی متن
def clean_text(text):
    # تبدیل به حروف کوچک
    text = text.lower()
    # حذف علائم نگارشی و کاراکترهای خاص
    text = re.sub(r'[^\w\s]', '', text)
    # حذف فاصله‌های اضافی
    text = ' '.join(text.split())
    return text

# اعمال پاک‌سازی روی ستون reviewText
df_3['reviewText'] = df_2['reviewText'].apply(clean_text)

# تبدیل برچسب‌ها (۱ تا ۵ به ۰ تا ۴)
df_3['overall'] = df_3['overall'] - 1

In [ ]:
df_3.summary = df_3.summary.fillna('neutral')
df_3['summary'] = df_3['summary'].apply(clean_text)


In [ ]:
## تقسیم داده های آموزشی و اعتبار سنجی
train_df, val_df = train_test_split(df_3, test_size=0.3, random_state=42)

# توکن‌سازی با توکنایزر Keras
vocab_size = 10000  # تعداد کلمات در واژگان
max_length = 512  # حداکثر طول توکن‌ها
oov_token = '<OOV>'  # توکن برای کلمات ناشناخته

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_df['reviewText'])

# تبدیل متن به توکن‌ها
train_sequences = tokenizer.texts_to_sequences(train_df['reviewText'])
val_sequences = tokenizer.texts_to_sequences(val_df['reviewText'])

In [ ]:
train_summary_sequences = tokenizer.texts_to_sequences(train_df['summary'])
val_summary_sequences = tokenizer.texts_to_sequences(val_df['summary'])

train_summary_padded = pad_sequences(train_summary_sequences, maxlen=128, padding='post', truncating='post')
val_summary_padded = pad_sequences(val_summary_sequences, maxlen=128, padding='post', truncating='post')

### Model

In [ ]:
# تعریف مدل Keras با دو ورودی (متن + ویژگی‌های اضافی + خلاصه متن)

# ورودی ویژگی‌های اضافی
asin_input = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='asin')
# تعداد دسته‌های منحصربه‌فرد برای Embedding
num_asin = len(le_asin.classes_)
## پردازش ویژگی های اضافی غیر ترتیبی
asin_embedding = tf.keras.layers.Embedding(input_dim=num_asin, output_dim=16)(asin_input)
asin_embedding = tf.keras.ops.squeeze(asin_embedding, axis=1)
# ترکیب ویژگی‌های اضافی
extra_dense = tf.keras.layers.Dense(64, activation='relu')(asin_embedding)

# ورودی متن
text_input = tf.keras.layers.Input(shape=(max_length,), name='text_input')
embedding = tf.keras.layers.Embedding(vocab_size, 128)(text_input)
lstm = tf.keras.layers.LSTM(64)(embedding)
text_output = tf.keras.layers.Dense(64, activation='relu')(lstm)

# ورودی خلاصه متن
summary_input = tf.keras.layers.Input(shape=(128,), name='summary_input')
embedding = tf.keras.layers.Embedding(vocab_size, 128)(text_input)
lstm = tf.keras.layers.LSTM(32)(embedding)
summary_output = tf.keras.layers.Dense(32, activation='relu')(lstm)

# ترکیب تمام ورودی‌ها
combined = tf.keras.layers.Concatenate()([text_output, extra_dense, summary_output])
dense = tf.keras.layers.Dense(64, activation='relu')(combined)
output = tf.keras.layers.Dense(5, activation='softmax')(dense)

In [ ]:
# تعریف مدل
model = tf.keras.Model(inputs=[text_input, asin_input, summary_input], outputs=output)

# کامپایل مدل
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_input          │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ asin (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 512, 128)  │  1,280,000 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 16)     │  1,451,440 │ asin[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     49,408 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ squeeze_1 (Squeeze) │ (None, 16)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      4,160 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,088 │ squeeze_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ summary_input       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ dense_2[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0],      │
│                     │                   │            │ summary_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     16,448 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 5)         │        325 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,802,869 (10.69 MB)

 Trainable params: 2,802,869 (10.69 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# آموزش مدل
history = model.fit(
    [train_padded, train_extra, train_summary_padded], train_labels,
    validation_data=([val_padded, val_extra, val_summary_padded], val_labels),
    epochs=7,
    batch_size=32,
    class_weight=class_weights
)

Epoch 1/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 466s 25ms/step - accuracy: 0.5071 - loss: 61.6872 - val_accuracy: 0.5801 - val_loss: 1.1307
Epoch 2/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 504s 25ms/step - accuracy: 0.5981 - loss: 7.7540 - val_accuracy: 0.6214 - val_loss: 1.0828
Epoch 3/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 492s 25ms/step - accuracy: 0.6409 - loss: 6.8508 - val_accuracy: 0.6450 - val_loss: 0.9358
Epoch 4/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 453s 25ms/step - accuracy: 0.6677 - loss: 6.3101 - val_accuracy: 0.6607 - val_loss: 0.8533
Epoch 5/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 517s 25ms/step - accuracy: 0.6795 - loss: 6.0005 - val_accuracy: 0.6710 - val_loss: 0.8531
Epoch 6/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 467s 25ms/step - accuracy: 0.6920 - loss: 5.7349 - val_accuracy: 0.6708 - val_loss: 0.8287
Epoch 7/7
18352/18352 ━━━━━━━━━━━━━━━━━━━━ 500s 25ms/step - accuracy: 0.6991 - loss: 5.5654 - val_accuracy: 0.6685 - val_loss: 0.9150
